In [4]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import RobustScaler
from xgboost import XGBRegressor
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import RobustScaler
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
# Load dataset
df = pd.read_csv('f1_2019_to_2023_all_drivers_all_data.csv', low_memory=False)

# Convert time columns to seconds
time_columns = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']
for col in time_columns:
    df[col] = pd.to_timedelta(df[col]).dt.total_seconds()

# Convert binary columns to integer type
df['Rainfall'] = df['Rainfall'].astype(int)
df['FreshTyre'] = df['FreshTyre'].astype(int)
df['IsAccurate'] = df['IsAccurate'].astype(int)


# Categorize weather condition based on centroid values of Kmeans clustering
def categorize_weather(row):
    if row['Rainfall'] > 0:
        return 'Rainy'
    elif row['AirTemp'] > 28.43213126:
        return 'high'
    elif row['AirTemp'] > 21.31279265:
        return 'medium'
    elif row['AirTemp'] > 12.84901403:
        return 'low'
    else:
        return 'very_low'

df['Weather_Category'] = df.apply(categorize_weather, axis=1)
df['Original_Weather_Category'] = df['Weather_Category']
df = pd.get_dummies(df, columns=['Weather_Category'])


# Keep the original Driver and Circuit for EDA
df['Original_Driver'] = df['Driver']

# Create Track temperature category based on the result of Kmeans clustering 
df['TrackTemp_Cat'] = pd.cut(df['TrackTemp'], bins=[0, 18.96764999, 27.87457484, 35.04425766, 41.75142602, 50.51006013], labels=['VERY_LOW', 'Low', 'Medium', 'Warm', 'High'])
df['Original_TrackTemp_Cat'] = df['TrackTemp_Cat']
df = pd.get_dummies(df, columns=['TrackTemp_Cat'])


#Feature Engineering with weather condition features
df['TrackConditionIndex'] = (df['AirTemp'] + df['TrackTemp'] + df['Humidity'] + df['Pressure'] +
                             df['WindSpeed'] + df['Rainfall']) / 6

# One-hot encoding
df = pd.get_dummies(df, columns=['Driver', 'Compound', 'Team','TrackStatus'])

# Drop irrelevant columns
columns_to_drop = ['Time', 'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
                   'PitOutTime', 'PitInTime', 'LapStartDate', 'Deleted', 'DeletedReason', 'FastF1Generated',
                   'IsPersonalBest', 'Sector3Time','LapStartTime','Sector2Time','Sector1Time','Circuit']
df.drop(columns=columns_to_drop, inplace=True)


# Select numerical values for scaling and imputation
numeric_features = ['Humidity', 'Pressure', 'WindDirection', 'WindSpeed','TrackTemp','AirTemp','TrackConditionIndex','Rainfall','SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST']


## Separate Rainy / dry days ##
# 1. Separate LapTime as dry or wet(rainy) condition ( since lapTime of rainy day would be recognized as outliers)
# 2. Remove Outliers for dry condition LapTime
# 3. Build Combined LapTime df (Outliers for dry days are deleted)

# Flag for rainy conditions
df['IsRainy'] = df['Rainfall'].apply(lambda x: 1 if x > 0 else 0)

# Separate dataframes for dry and wet conditions
df_dry = df[df['IsRainy'] == 0]
df_wet = df[df['IsRainy'] == 1]

# Remove Outliers for dry days using the IQR method that effective to removes extreme outliers
Q1_dry = df_dry['LapTime'].quantile(0.25)
Q3_dry = df_dry['LapTime'].quantile(0.75)
IQR_dry = Q3_dry - Q1_dry
lower_bound_dry = Q1_dry - 1.5 * IQR_dry
upper_bound_dry = Q3_dry + 1.5 * IQR_dry
df_dry_filtered = df_dry[(df_dry['LapTime'] >= lower_bound_dry) & (df_dry['LapTime'] <= upper_bound_dry)]
df_dry_filtered = df_dry_filtered.copy()
df_wet = df_wet.copy()


from fancyimpute import IterativeImputer as MICE


mice_imputer = MICE()
df_imputed = mice_imputer.fit_transform(df)



## Scaling to the selected numeric features ##
robust_scaler = RobustScaler()

# Applying scaling to the numeric features
df_dry_filtered[numeric_features] = robust_scaler.fit_transform(df_dry_filtered[numeric_features])
df_wet[numeric_features] = robust_scaler.transform(df_wet[numeric_features])  # Use transform, not fit_transform

# Combining the datasets after scaling
df_combined = pd.concat([df_dry_filtered, df_wet], ignore_index=True)


# Remove rows where 'Position' column contains NaN
df_combined['LapTime'].fillna(method='ffill', inplace=True)

df_combined['Position'].fillna(df_combined['Position'].mean(), inplace=True)

df_combined.info(verbose=True)


# Drop Strings columns
df_combined = df_combined.drop(['Original_Driver', 'Original_Weather_Category', 'Original_TrackTemp_Cat'], axis=1)
# Define function to compare the impact of different imputation methods
def model_performance_comparison(df, numeric_features, target_column):
    # Prepare data for modeling
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the imputers
    imp_iterative = IterativeImputer(random_state=42)
    imp_mean = SimpleImputer(strategy='mean')

    # Fit and transform the training data using both imputers
    X_train_iterative = imp_iterative.fit_transform(X_train[numeric_features])
    X_train_mean = imp_mean.fit_transform(X_train[numeric_features])

    # Prepare complete training dataframes including non-numeric features
    X_train_iterative = np.concatenate((X_train_iterative, X_train.drop(columns=numeric_features).values), axis=1)
    X_train_mean = np.concatenate((X_train_mean, X_train.drop(columns=numeric_features).values), axis=1)

    # Transform the test data (do not fit the imputer to avoid data leakage)
    X_test_iterative = imp_iterative.transform(X_test[numeric_features])
    X_test_mean = imp_mean.transform(X_test[numeric_features])
    X_test_iterative = np.concatenate((X_test_iterative, X_test.drop(columns=numeric_features).values), axis=1)
    X_test_mean = np.concatenate((X_test_mean, X_test.drop(columns=numeric_features).values), axis=1)

    # Define and fit the XGBoost model on both datasets
    model_iterative = XGBRegressor(random_state=42)
    model_mean = XGBRegressor(random_state=42)
    model_iterative.fit(X_train_iterative, y_train)
    model_mean.fit(X_train_mean, y_train)

    # Make predictions on the test set
    y_pred_iterative = model_iterative.predict(X_test_iterative)
    y_pred_mean = model_mean.predict(X_test_mean)

    # Calculate performance metrics
    mse_iterative = mean_squared_error(y_test, y_pred_iterative)
    mse_mean = mean_squared_error(y_test, y_pred_mean)
    r2_iterative = r2_score(y_test, y_pred_iterative)
    r2_mean = r2_score(y_test, y_pred_mean)

    # Print the performance metrics
    print("Iterative Imputation MSE:", mse_iterative, "R2:", r2_iterative)
    print("Mean Imputation MSE:", mse_mean, "R2:", r2_mean)

# Example usage
model_performance_comparison(df_combined, numeric_features, 'LapTime')


/var/folders/_5/533f3cq93vq25vl2f3txpkb40000gn/T/ipykernel_15899/382747998.py:125: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_combined['LapTime'].fillna(method='ffill', inplace=True)
/var/folders/_5/533f3cq93vq25vl2f3txpkb40000gn/T/ipykernel_15899/382747998.py:125: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_combined['LapTime'].fillna(method='ffill', inplace=True)
/var/folders/_5/533f3cq93vq25vl2f3txpkb40000gn/T/ipykernel_15899/

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96556 entries, 0 to 96555
Data columns (total 112 columns):
 #    Column                     Dtype   
---   ------                     -----   
 0    DriverNumber               int64   
 1    LapTime                    float64 
 2    LapNumber                  float64 
 3    Stint                      float64 
 4    SpeedI1                    float64 
 5    SpeedI2                    float64 
 6    SpeedFL                    float64 
 7    SpeedST                    float64 
 8    TyreLife                   float64 
 9    FreshTyre                  int64   
 10   Position                   float64 
 11   IsAccurate                 int64   
 12   AirTemp                    float64 
 13   Humidity                   float64 
 14   Pressure                   float64 
 15   Rainfall                   float64 
 16   TrackTemp                  float64 
 17   WindDirection              float64 
 18   WindSpeed                  float64 
 19   Ye

ImportError: IterativeImputer is experimental and the API might change without any deprecation cycle. To use it, you need to explicitly import enable_iterative_imputer:
from sklearn.experimental import enable_iterative_imputer